# Conformal Prediction

**Sources**: <br>

[Angelopoulos, A. N., & Bates, S. (2021). A gentle introduction to conformal prediction and distribution-free uncertainty quantification. arXiv preprint arXiv:2107.07511](https://arxiv.org/abs/2107.07511)

[Conformal Prediction Tutorial](https://github.com/aangelopoulos/conformal-prediction)

## Conformal Prediction Part 1: Basic Understanding

### General Understanding

**Conformal Prediction** 

Conformal Prediction can be used with any pre-trained model, such as a neural network. It aims to predict an interval of target value instead of a point-wise prediction.It works perfectly well with the concept of **uncertainty**.

*What we need*:
- A *Dataset* : $\{x_i,y_i\}^n_{i=1}$, which is independent and identically distributed (i.i.d).
- A *Model* : $\hat{\pi}_y(x)$, $\pi_y(x) = \mathbb{P}[Y=y | X=x]$
- A *New Sample* : $X_{n+1}$

*What we generate*:
- A Set of prediction $T(x_{n+1})  \subseteq y$, which contains the true class $y_{n+1}$ with high $\mathbb{P}$ (probability)

**Coverage**

Coverage means that $\mathbb{P}[y_{n+1} \in T(x_{n+1})] \geq 1-\alpha$. The probability of the label is in the prediction set generated by the the new sample $x_{n+1}$ is greater than $1-\alpha$, where $\alpha$ is a hyper-parameter error rate.'

**Goal**
- Exact Coverage: $|T(x_{n+1})| = 1$
- Small Set: $|T(x_{n+1})| < n$
- "Adaptive": $|T(x_{n+1})|$ smaller for easy task and $|T(x_{n+1})| is bigger for hard task$

**Property**
- Any Model
- Any Dataset
- Score Function Matters (Important Decision)

**Conformal Prediction (General Case)**
1) Identify a heuristic notion of uncertainty
2) Define a scalar score function $s(x,y) \in R$
3) Compute $\hat{q} = \frac{\lceil (n+1)(1-\alpha)\rceil}{n}$ is the quantile of $s(x_1,y_1),\cdots, s(x_n),y_n$ (ca)
4) Deploy: $T(x) = \{y: s(x,y) \leq \hat{q}\}$

### Theorem 1 [Coverage]

$1-\alpha \leq \mathbb{P}[y_{n+1} \in T(x_{n+1})] \leq 1-\alpha +\frac{1}{n+1}$

This can be applied to any algorithm, any dataset.


*Why it works?*  

Symmetry!!! The probability of x3 falls on the left of  q is greater than $1-\alpha$

---x2-------------x1----------q--------

### Methods 1 (Vovk et al)

What if we learn a rule to predict sets? using calbration dataset $\{x_i,y_i\}^{n}_{i=1}$

1. Get estimated score of the correct classes ($y_i$) for each of $(x_i,y_i)$: $\{E_{i}\}^{n}_{i=1}$
2. Take the 10% quantile: $\hat{q}$ - At least 90% of examples have true class score above $\hat{q}$
```
q_hat = np.quantile([E1,...En],0.1,'lower')
```

3. Form prediction sets: {All classes whose score exceeds $\hat{q}$ when $x_{n+1}$ input} = Valid Prediction Set! $T(x_{n+1})$

### Method 2 (Romano et al)

|       Method 1       |   Method 2|
|----------------------|-----------|
|smallest average size | usually larger size|
| not very adaptive | designed to be adaptive|
|only use output of true class | use output of all classes|

#### Conformalized Quantile **(Classification)**

1) Get a score of the correct class
> Sort the softmax estimation. $E_i = \sum_{j=1}^k \hat{\pi}(x_i)_{(j)}$ where k is the rank of true class. Sum up all scores from high to low until the score of the true label is included.

2) Take the 90% quantile
>```q_hat = np.quantile([E1,...En],0.9,'upper')```

3) Form prediction sets:
> {The K most likely classes where $\sum_{j=1}^K \hat{\pi}(x_{n+1})_{y_{n+1}} \geq \hat{q}$ = Valid Prediction Set $T(x_{n+1})$



#### Conformalized Quantile **(Regression)**

We have two models $\hat{t}_{\alpha/2}(x)$ and$\hat{t}_{1-\alpha/2}(x)$, which is 5% quantile and 95% quantile. This can be achieved by training NN with [pinball loss](https://www.lokad.com/pinball-loss-function-definition/).

> *pinball* loss, also referred to as the quantile loss, can be used to assess the accuracy of a quantile forecast.
> 
> $\begin{align}L_{\tau}(y,z) & = (y-z)\tau & \text{ if } y\geq z\\
        &= (z-y)(1-\tau) & \text{ if } z > y
        \end{align} $
> 
> where $\tau$ is the target quantile, $y$ is the real value and $z$ is the quamtile forecast
>
> The pinball loss is always positive. The larger the value of *pinball* loss, the further away from the target $y$. The lower the pinball loss, the more accurate the quantile forecast.

<div style="text-align:center;">
  <img height="100%" width="50%" src="sources/conformalized_quantile_regression.png" />
</div>

1) Get score of correct class
> $E_i$ = Projection of $y_i$ onto $[\hat{t}_{\alpha/2}(x_i),\hat{t}_{1-\alpha/2}(x_i)]$, or distance of how far the estimation is outside of the band.

2) Take the 90% quantile
>```q_hat = np.quantile([E1,...En],0.9,'upper')```

3) Form prediction sets
>$T(x_{n+1})$ = Valid Prediction Set $T(x_{n+1}) = [\hat{t}_{\alpha/2}(x_{n+1})-\hat{q},\hat{t}_{1-\alpha/2}(x_{n+1})+\hat{q}]$ = Valid Prediction Set! $T(x_{n+1})$


## Conformal Prediction Part 2: Conditional Coverage and Diagnostics

### Recap + Intro to Conditional

**Gurantee**:

For any model distribution, $n$,$\alpha$, 
$$\mathbb{P}[y \in T(x)] \geq 1-\alpha$$

which satisfied "marginal" coverage - it means that on average I have $1-\alpha$ coverage. Please refer to Theorem 1 above.

However, the errors can be systematic, which means that it makes mistake for any samples fill into group A but no mistake for samples in group B. Therefore, we need "conditional" (stronger) coverage of conformal prediction.

<div style="text-align:center;">
  <img height="100%" width="40%" src="sources/cpmformal_part2_marginal_conditional.png" />
 <img height="100%" width="40%" src="sources/conformal_part2_marginal_conditional2.png" />
</div>

With **No Coverage**, we could have 50% errors in Group 1 and even more in Group 2.

At least **Marginal Coverage** with conformal, where, on average, we have $1-\alpha = 90\%$ coverage. In the image above, we have 100% coverage for group 1 and 80% coverage for group 2; therefore, we will have 90% on average.

However, we would prefer the **Conditional Coverage** where we have 90% coverage on Group 1 and 90% coverage on Group 2.

### Theorem 1 [Coverage]

For any model distribution, $n$,$\alpha$, 
$$\mathbb{P}[y \in T(x) | X= x] \geq 1-\alpha$$


Conformal does NOT gurantee this but we can usually get it with **techniques** like Conformalized Quantile Regression (CQR), APS

#### Example **(Classification)**

If we knew $\mathbb{P}[Y=y | X= x]$, then we would achive conditional converage. In which case, we count down until the sum of probabilities reach 90%. With this strategy, we would achieve conditional converage. APS: use it as the score function

### Evaluating Conformal

1) Coverage
2) Set Size
3) Adaptiveness (conditional coverage)

#### 1) Coverage

Coverage is used to debug.

We know the exact distribution. Bug check. 

$$Coverage \approx Beta(l,n-l+1), l = \lceil (n+1)(1-\alpha)\rceil$$

For T trials:
1) resample data
2) calculate coverage on the validation set

plot histogram of empirical coverage. Empirical coverage should be similar to theoretical coverage
<div style="text-align:center;">
  <img height="100%" width="50%" src="sources/conformal_part2_coverage.png" />
</div>

#### 2) Set Size

- Small is generally better
- Small variance can be a warning sign
- Similar evaluation as before

#### 3) Adaptiveness (conditional coverage)

- Often impossible to measure directly (if X or y are continuous)
- Proxies:
    - Label-Stratefied Coverage: Evaluate the Coverage of each class; accuracy close to each other would mean good coverage.
    - Size-Stratefied Coverage: Coverage Grouped by Label Size, such as small sets, large sets

#### Exapmles: Efficiently Evaluating Conformal **(Classification)**

Model: $\hat{f}: X \rightarrow \Delta_k$ (pretrained)

Dataset: {$(x_1,y_1), \cdots, (x_{n+n_{val}},y_{n+n_{val}})$}

Step 0: cache matrix

D = $\begin{bmatrix} \hat{f}(x_1) \\ \vdots \\ \hat{f}(x_{n+n_{val}}) \end{bmatrix}$

where $\hat{f}(x_1)$ is a softmax vector in classification cases

<div style="text-align:center;">
  <img height="100%" width="80%" src="sources/conformal_part2_algorithm.png" />
</div>

Benefit: Run the model once.

## Conformal Prediction Part 3: Beyond Classification/Regression 

There could be more applications with conformal, such as Tumor Segmentation, OOD detection + Coverage, Instance Segmentation

### Tumor Segmentation

Risk Function:
$$R(\lambda) = \mathbb{E}[1-\frac{\#(y \cap T_{\lambda}(x))}{\#(y)}]$$

where $y \cap T_{\lambda}(x)$ is the False Negative Rate. $\frac{\#(y \cap T_{\lambda}(x))}{\#(y)}$  is the False Negative Porportion.

$R(\lambda)$ is a Risk. (e.g. $\mathbb{E}[FNP(T_{\lambda}(x),y)]$). We want to find $\lambda$ so that $R(\lambda) \leq \alpha$

<div style="text-align:center;">
  <img height="100%" width="80%" src="sources/conformal_part3_tumor_risk.png" />
</div>

#### **Learn then Test**: Real Procedure

1) Discretize  into $\Lambda = \{\lambda_1, \cdots, \lambda_N\}$
2) p-value (via concentration), $H_j: R(\lambda_j) > \alpha   \rightarrow \{p_1,\cdots,p_N\}$
> The Null hypothesis is the risk at $\lambda_j$ is not controled as it is larger than $\alpha$
> Then we get all N p-values. When p-value is small, it means that we rejected the hypothesis, which means the risk is controlled.
> p-value small means the risk is controlled at that point

3) Multiple testing, e.g. $\hat{\Lambda} = \{\lambda_j: p_j \leq \frac{\delta}{N}\}$

##### **2) Getting p-values**

This means that the risk is not controlled at $\lambda_j$. p-value is "strength of evidence". p-value is superuniform under $H_0$
$$H_j: R(\lambda_j) > \alpha   \rightarrow \{p_1,\cdots,p_N\}$$

$$\mathbb{P}[\hat{R}(\lambda) - R(\lambda) \geq t] \leq e^{-\frac{1}{2}nt^2}$$
This is known as Hoeffdig p-values: $e^{-\frac{1}{2}n(\alpha - \hat{R}(\lambda))^2_{+}}$ for bounded losses

##### **3) Multiple testing**



$\delta$ level $\rightarrow$ $N \times \delta$ False Discoveries. Therefore, we use $\frac{\delta}{N}$

<div style="text-align:center;">
  <img height="100%" width="80%" src="sources/conformal_part3_multipletesting.png" />
</div>


### Out-of Distribution Detection

<div style="text-align:center;">
  <img height="100%" width="80%" src="sources/conformal_part3_OOD1.png" />
</div>



**Setting**: X is image, y is $\{1,...,K\}$, $\hat{f}_y(x)$ is a classifier

**Risks**:$R_1(\lambda) = \mathbb{P}[T_{\lambda}(x) = \emptyset]$,$R_2(\lambda) = \mathbb{P}[Y \notin T_{\lambda}(x)]$

**Set Construction**:
$$T_{\lambda}(x) = \begin{cases} \emptyset & \text{if }max_y \hat{f}_y(x) < \lambda \\ \{y_: \hat{f}_y(x) \geq \lambda_2\} & else \end{cases}$$

If none of the classes in my prediction returns a higher than $\lambda_1$ probability, then I will return $\emptyset$. Otherwise, I will return my confidence set, including all labels above $\lambda_2$. This strategy has 2 variables $\lambda_1,\lambda_2$.